# Genius Lyrics About Section
##### Web scraping the about sections from Genius Lyrics for artists' most popular songs

### Import Statements

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Function to scrape the 'About' section for songs

In [2]:
def find_about(url):
    response = requests.get(url)
    # Return None if you get any error (200 means successful request)
    if response.status_code != 200:
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    # About section classes start with "RichText__Container-oz284w-0"
    about_section = soup.find_all('div', class_='RichText__Container-oz284w-0')
    
    # Getting the text from the about sections 
    if about_section:
        text = about_section[0].get_text(strip=True)
    else:
        text = None
    
    return text

### Function to find an artist's most popular songs

In [3]:
def song_names(artist):
    format_artist = artist.lower().replace(" ", "-")
    url_artist = f"https://genius.com/artists/{format_artist}"
    song_list = []
    response = requests.get(url_artist)
    
    # Return None if you get any error (200 means successful request)
    if response.status_code != 200:
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
        # About section classes are titled "mini_card-title"
    song_titles = soup.find_all('div', class_='mini_card-title', limit=11)
    
    for title in song_titles:
        song_text = title.get_text(strip=True)
        song_list.append({"song": song_text, "artist": artist.lower()})
    
    return song_list

### Running the code

In [4]:
def main():
    count = int(input("How many artists would you like to learn about? "))
    data = [] 

    for _ in range(count):
        artist = input("Enter the artist's name: ").strip()
        top_songs = song_names(artist)

        if top_songs is None:
            print(f"Could not retrieve songs for {artist}.")
            continue

        for song_info in top_songs:
            format_artist = song_info['artist'].replace(" ", "-")
            format_song = song_info['song'].lower().replace(" ", "-")
            url = f"https://genius.com/{format_artist}-{format_song}-lyrics"
            
            text = find_about(url)

            if text is None:
                print(f"The 'About' section for '{song_info['song']}' by '{artist}' could not be found.")
            else:
                data.append({
                    "artist": artist.capitalize(),
                    "song": song_info['song'].capitalize(),
                    "about": text
                })
    
    # Create DataFrame and export as CSV
    df = pd.DataFrame(data)
    df = df.sort_values(by=['artist', 'song'])
    print(df)
    df.to_csv('song_information.csv', index=False, encoding='utf-8')

if __name__ == "__main__":
    main()

ValueError: invalid literal for int() with base 10: ''